In [22]:
import tqdm
# Autoreload possibly interferes with IntelliJ debugging
%reload_ext autoreload
%autoreload 2
import logging
class FlushHandler(logging.StreamHandler):
    def emit(self, record):
        super().emit(record)
        self.flush()

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[FlushHandler()])
log = lambda msg: logging.info(msg)


# Full pipeline (multiple files)

In [23]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data-day"
stats_df = pd.read_csv(input_dir + os.path.sep + "day_stats.csv")
# stats_df = pd.read_csv("C:\\dev\\play\\brainwave-data\\stats.csv")

In [24]:
from models.eeg_states.eeg_states import load_and_prepare_day_data_energy_eeg_state_events

events = load_and_prepare_day_data_energy_eeg_state_events()

C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:115: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-11-15 09:34:49.320000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:117: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']


## Convert Brainflow files to FIF

In [25]:
def get_brainflow_compressed_filename(full_input_filename: str) -> str:
    full_output_dirname = webserver.output_dirname(full_input_filename)
    compressed_full_output_filename = str(os.path.join(full_output_dirname, os.path.basename(full_input_filename))) + '.bz2'
    return compressed_full_output_filename

In [43]:
from datetime import datetime

import webserver
import convert
# import zstandard as zstd
import os
import bz2
import time
import shutil
from tqdm.notebook import trange, tqdm

errors = []
processed = []

# Could get these working later
skip_list = []

force = False

def compress_bz2(input_file, output_file):
    start_time = time.time()
    with open(input_file, 'rb') as f_in:
        with bz2.open(output_file, 'wb', compresslevel=9) as f_out:
            shutil.copyfileobj(f_in, f_out)
    end_time = time.time()
    return end_time - start_time, os.path.getsize(output_file)

for root, dirs, files in os.walk(input_dir):
    # Exclude the last file, which we assume to be the most recent, and possibly still being written
    files = [file for file in files if file.endswith(".brainflow.csv")][:-1]
    for idx, file_name in tqdm(enumerate(files), desc="Processing directories", total=(len(files))):  
        full_input_filename = os.path.join(root, file_name)
        try:
            full_output_dirname = webserver.output_dirname(full_input_filename)
            full_output_filename = str(os.path.join(full_output_dirname, 'raw.fif'))
            
            compressed_full_output_filename = get_brainflow_compressed_filename(full_input_filename)
            
            if not os.path.exists(compressed_full_output_filename) or force:
                log(f"Compressing file {full_input_filename} to " + compressed_full_output_filename)
                processed.append("Compressing " + full_input_filename)
                try:
                    os.mkdir(os.path.dirname(compressed_full_output_filename))
                except:
                    pass
                compress_bz2(full_input_filename, compressed_full_output_filename) 
                
            if os.path.exists(full_output_filename) and not force:
                log(f"Skipping file {full_input_filename} as {full_output_filename} and {compressed_full_output_filename} already exist")
                continue
            should_skip = False
            for s in skip_list:
                if s in full_input_filename:
                    log(f"Skipping file {full_input_filename}")
                    should_skip = True
            if not should_skip:
                log(f"Processing file {full_input_filename}")
                processed.append("Processing " + full_input_filename)
                channels = ['Fpz-M1']
                date_time_str = os.path.basename(full_input_filename).removesuffix(".brainflow.csv")
                date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
    
                if channels is not None:
                    log(f"Processing file {full_input_filename} with channels {channels}")
                    convert.convert_and_save_brainflow_file_with_gap_filling(log, full_input_filename, full_output_filename, channels)

        except Exception as e:
            msg = "Error processing file: " + full_input_filename
            log(msg)
            log(e)
            errors.append(msg)


Processing directories:   0%|          | 0/7 [00:00<?, ?it/s]

2024-11-16 16:31:15,731 - INFO - Compressing file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50.brainflow.csv to C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\2024-11-15-09-10-50.brainflow.csv.bz2
2024-11-16 16:31:20,188 - INFO - Processing file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50.brainflow.csv
2024-11-16 16:31:20,189 - INFO - Processing file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50.brainflow.csv with channels ['Fpz-M1']
2024-11-16 16:31:20,337 - INFO - Memory Usage: 708.99 MB GC to 708.99 MB
2024-11-16 16:31:20,337 - INFO - Reading Brainflow file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50.brainflow.csv
2024-11-16 16:31:22,201 - INFO - Finished reading Brainflow file
2024-11-16 16:31:22,332 - INFO - Memory Usage: 739.95 MB GC to 739.95 MB
2024-11-16 16:31:22,333 - INFO - EEG channels: [1, 2, 3, 4, 5, 6, 7, 8]
C:\dev\play\brainwave-processor\convert.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Gaps:  datetime
False    169061
Name: count, dtype: int64



Processing rows: 100%|██████████| 169060/169060 [00:00<00:00, 290692.94it/s]
2024-11-16 16:31:23,049 - INFO - Initial timestamp: 2024-11-15 09:10:50.479561090+00:00 from 1731661850.479561
2024-11-16 16:31:23,051 - INFO - Info <Info | 7 non-empty values
 bads: []
 ch_names: Fpz-M1
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: 2024-11-15 09:10:50 UTC
 nchan: 1
 projs: []
 sfreq: 250.0 Hz
>
2024-11-16 16:31:23,181 - INFO - Memory Usage: 751.57 MB GC to 750.28 MB
2024-11-16 16:31:23,182 - INFO - Saving to C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.fif
2024-11-16 16:31:23,203 - INFO - Compressing file C:\dev\play\brainwave-data-day\2024-11-15-09-22-47.brainflow.csv to C:\dev\play\brainwave-data-day\2024-11-15-09-22-47\2024-11-15-09-22-47.brainflow.csv.bz2
2024-11-16 16:31:24,331 - INFO - Processing file C:\dev\play\brainwave-data-day\2024-11-15-09-22-47.brainflow.csv
2024-11-16 16:31:24,332 - INFO - Processing file C:\dev\play\brainwave

Gaps:  datetime
False    44693
True         2
Name: count, dtype: int64


...

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

In [44]:

errors

[]

In [45]:
processed

['Compressing C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-10-50.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-10-50.brainflow.csv',
 'Compressing C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-22-47.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-22-47.brainflow.csv',
 'Compressing C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-26-33.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-26-33.brainflow.csv',
 'Compressing C:\\dev\\play\\brainwave-data-day\\2024-11-16-10-59-45.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data-day\\2024-11-16-10-59-45.brainflow.csv',
 'Compressing C:\\dev\\play\\brainwave-data-day\\2024-11-16-14-12-44.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data-day\\2024-11-16-14-12-44.brainflow.csv',
 'Compressing C:\\dev\\play\\brainwave-data-day\\2024-11-16-14-16-36.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data-day\\2024-11-16-14-16-36.br

## Run pipeline on FIF files

In [46]:
import contextlib
import io
import run_day_pipeline
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import trange, tqdm

errors = []
dataframes = []

# Could get these working later
skip_list = []

def process_file(root, dir_name):
    input_file = os.path.join(root, dir_name, "raw.fif")
    if dir_name in skip_list:
        log(f"Skipping {dir_name}: " + input_file)
        return None
    try:
        log(f"Processing file: " + input_file)
        if os.path.exists(input_file):
            yasa_df = run_day_pipeline.cached_pipeline(log, input_file, stats_df, events)
            #log(f"Returning {yasa_df.head()}")
            return yasa_df
    except Exception as e:
        msg = f"Error processing file: " + input_file + " - " + str(e)
        log(msg)
        errors.append(msg)
        log(e)
    return None

for root, dirs, files in os.walk(input_dir):
    for dir_name in tqdm(dirs, desc="Processing directories", total=len(dirs)):
        output_buffer = io.StringIO()
        with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
            df = process_file(root, dir_name)
            if df is not None:
                dataframes.append(df)

all = pd.concat(dataframes)
log(f"Finished processing, have {len(all)} files in total")

Processing directories:   0%|          | 0/7 [00:00<?, ?it/s]

2024-11-16 16:36:29,178 - INFO - Processing file: C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.fif
2024-11-16 16:36:29,184 - INFO - Loading cached file C:\dev\play\brainwave-data-day\2024-11-15-09-10-50\raw.output.csv
2024-11-16 16:36:29,213 - INFO - Processing file: C:\dev\play\brainwave-data-day\2024-11-15-09-22-47\raw.fif
2024-11-16 16:36:29,223 - INFO - Loading cached file C:\dev\play\brainwave-data-day\2024-11-15-09-22-47\raw.output.csv
2024-11-16 16:36:29,248 - INFO - Processing file: C:\dev\play\brainwave-data-day\2024-11-15-09-26-33\raw.fif
2024-11-16 16:36:29,258 - INFO - Loading cached file C:\dev\play\brainwave-data-day\2024-11-15-09-26-33\raw.output.csv
2024-11-16 16:36:29,392 - INFO - Processing file: C:\dev\play\brainwave-data-day\2024-11-16-10-59-45\raw.fif
2024-11-16 16:36:29,399 - INFO - Loading cached file C:\dev\play\brainwave-data-day\2024-11-16-10-59-45\raw.output.csv
2024-11-16 16:36:29,480 - INFO - Processing file: C:\dev\play\brainwave-data-day\2024-11

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

Processing directories: 0it [00:00, ?it/s]

2024-11-16 16:36:29,930 - INFO - Finished processing, have 918 files in total


In [47]:
dataframes

[    epoch                          TimestampUK  Fpz-M1_eeg_abspow  \
 0       0  2024-11-15 09:10:50.479561090+00:00       7.072715e-09   
 1       1  2024-11-15 09:11:20.479561090+00:00       2.076430e-09   
 2       2  2024-11-15 09:11:50.479561090+00:00       4.383291e-09   
 3       3  2024-11-15 09:12:20.479561090+00:00       1.534609e-09   
 4       4  2024-11-15 09:12:50.479561090+00:00       3.456014e-09   
 5       5  2024-11-15 09:13:20.479561090+00:00       1.229371e-09   
 6       6  2024-11-15 09:13:50.479561090+00:00       1.208887e-09   
 7       7  2024-11-15 09:14:20.479561090+00:00       6.455377e-09   
 8       8  2024-11-15 09:14:50.479561090+00:00       6.425787e-09   
 9       9  2024-11-15 09:15:20.479561090+00:00       5.458992e-09   
 10     10  2024-11-15 09:15:50.479561090+00:00       3.031416e-09   
 11     11  2024-11-15 09:16:20.479561090+00:00       2.131056e-09   
 12     12  2024-11-15 09:16:50.479561090+00:00       3.173361e-09   
 13     13  2024-11-

In [48]:
errors

[]

# Recalculate scalings
N.b. can be run frequently but will only be picked up by new runs.  Maybe worth occasionally regenerating all old files.
And yes, for new features have to rerun the pipeline on everything, then generate the stats here, then rerun the pipeline again on everything to have them use those.

In [49]:
from scaling import only_eeg

only_eeg_cols = list(only_eeg(all).columns)
assert any(col.startswith("Main") for col in only_eeg_cols), "No column starting with 'Main' found in only_eeg_cols"

In [50]:
import scaling

stats = scaling.stats(all)
stats.to_csv(input_dir + "/day_stats.csv")
assert any(stats['Column'].str.startswith("Main")), "No row starting with 'Main' found in column_name"
stats

C:\Users\graha\.conda\envs\tf\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\graha\.conda\envs\tf\lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\graha\.conda\envs\tf\lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\graha\.conda\envs\tf\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\graha\.conda\envs\tf\lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\graha\.conda\envs\tf\lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in sub

,Column,Mean,P10,P90,Min,Max,StdDev
0,Fpz-M1_eeg_abspow,2.512660e-09,7.820518e-10,4.690927e-09,4.706757e-11,1.712247e-08,1.645689e-09
1,Fpz-M1_eeg_abspow_c7min_norm,3.375338e-02,-2.996965e-01,4.472095e-01,-8.923340e-01,1.359026e+00,3.098646e-01
2,Fpz-M1_eeg_abspow_p2min_norm,5.648157e-02,-2.795780e-01,4.616640e-01,-9.464824e-01,1.303757e+00,3.153609e-01
3,Fpz-M1_eeg_alpha,1.727805e-02,9.531571e-03,2.565649e-02,3.557820e-03,1.783833e-01,1.006551e-02
4,Fpz-M1_eeg_alpha_c7min_norm,3.631678e-02,-2.421503e-01,3.584531e-01,-1.005664e+00,1.396771e+00,3.021770e-01
...,...,...,...,...,...,...,...
427,Main_eeg_thetaabsaa_c7min_norm_s,6.406378e-01,-4.367109e-02,1.359958e+00,-7.330960e-01,4.909689e+00,6.342417e-01
428,Main_eeg_thetaabsaa_p2min_norm_s,5.022572e-01,2.950210e-02,1.021727e+00,-7.481929e-01,3.518574e+00,4.682687e-01
429,Main_eeg_thetaabsab_s,4.722393e-01,5.651317e-02,9.941704e-01,-1.187585e-01,3.752515e+00,3.912538e-01
430,Main_eeg_thetaabsab_c7min_norm_s,6.340361e-01,8.132866e-02,1.317985e+00,-8.841956e-01,2.545987e+00,4.997932e-01


# Upload to GCS

In [51]:
from upload import upload_dir_to_gcs_skipping_existing
import os

errors = []
dataframes = []

for root, dirs, files in os.walk(input_dir):
    for dir_name in tqdm(reversed(dirs), desc="Uploading directories", total=len(dirs)):
        full_dir_name = os.path.join(root, dir_name)
        try:
            upload_dir_to_gcs_skipping_existing(log, 'examined-life-input-eeg-day', full_dir_name, dir_name)
        except Exception as e:
            log("Error processing file: " + input_dir)
            log(e)

for error in errors:
    log(error)

log("All uploaded")

Uploading directories:   0%|          | 0/7 [00:00<?, ?it/s]

2024-11-16 16:36:41,955 - INFO - Skipping upload of C:\dev\play\brainwave-data-day\2024-11-16-16-11-03, identical directory already exists in GCS.
2024-11-16 16:36:44,825 - INFO - Skipping C:\dev\play\brainwave-data-day\2024-11-16-14-16-36\2024-11-16-14-16-36.brainflow.csv.bz2, identical file already exists in GCS.
2024-11-16 16:36:45,251 - INFO - Skipping C:\dev\play\brainwave-data-day\2024-11-16-14-16-36\raw.edf, identical file already exists in GCS.
2024-11-16 16:36:45,423 - INFO - C:\dev\play\brainwave-data-day\2024-11-16-14-16-36\raw.fif uploading to 2024-11-16-14-16-36/raw.fif, does_not_exist=False has_changed=True
2024-11-16 16:36:52,009 - INFO - Skipping C:\dev\play\brainwave-data-day\2024-11-16-14-16-36\raw.output.csv, identical file already exists in GCS.
2024-11-16 16:36:52,465 - INFO - Uploaded directory checksum for C:\dev\play\brainwave-data-day\2024-11-16-14-16-36 to 2024-11-16-14-16-36/directory_checksum.md5
2024-11-16 16:36:55,263 - INFO - Skipping C:\dev\play\brainwav

Uploading directories: 0it [00:00, ?it/s]

Uploading directories: 0it [00:00, ?it/s]

Uploading directories: 0it [00:00, ?it/s]

Uploading directories: 0it [00:00, ?it/s]

Uploading directories: 0it [00:00, ?it/s]

Uploading directories: 0it [00:00, ?it/s]

Uploading directories: 0it [00:00, ?it/s]

2024-11-16 16:37:33,219 - INFO - All uploaded


# Check if can delete Brainwave files that are safely backed up

In [35]:

import os

errors = []
can_delete = []
cannot_delete = []

for root, dirs, files in os.walk(input_dir):
    for idx, file_name in enumerate(files):
        full_input_filename = os.path.join(root, file_name)
        if full_input_filename.endswith(".brainflow.csv"):
            compressed_full_output_filename = get_brainflow_compressed_filename(full_input_filename)
                
            if os.path.exists(compressed_full_output_filename):
                can_delete.append({
                    'backed_up': compressed_full_output_filename,
                    'full_filename': full_input_filename
                })
            else:
                cannot_delete.append(full_input_filename)
    

In [36]:
can_delete_df = pd.DataFrame(can_delete)
can_delete_df

,backed_up,full_filename
0,C:\dev\play\brainwave-data-day\2024-11-15-09-1...,C:\dev\play\brainwave-data-day\2024-11-15-09-1...
1,C:\dev\play\brainwave-data-day\2024-11-15-09-2...,C:\dev\play\brainwave-data-day\2024-11-15-09-2...
2,C:\dev\play\brainwave-data-day\2024-11-15-09-2...,C:\dev\play\brainwave-data-day\2024-11-15-09-2...
3,C:\dev\play\brainwave-data-day\2024-11-16-10-5...,C:\dev\play\brainwave-data-day\2024-11-16-10-5...
4,C:\dev\play\brainwave-data-day\2024-11-16-14-1...,C:\dev\play\brainwave-data-day\2024-11-16-14-1...
5,C:\dev\play\brainwave-data-day\2024-11-16-14-1...,C:\dev\play\brainwave-data-day\2024-11-16-14-1...
6,C:\dev\play\brainwave-data-day\2024-11-16-16-1...,C:\dev\play\brainwave-data-day\2024-11-16-16-1...


In [37]:
cannot_delete

['C:\\dev\\play\\brainwave-data-day\\2024-11-16-16-12-49.brainflow.csv']

In [38]:
can_delete_df['full_filename']

0    C:\dev\play\brainwave-data-day\2024-11-15-09-1...
1    C:\dev\play\brainwave-data-day\2024-11-15-09-2...
2    C:\dev\play\brainwave-data-day\2024-11-15-09-2...
3    C:\dev\play\brainwave-data-day\2024-11-16-10-5...
4    C:\dev\play\brainwave-data-day\2024-11-16-14-1...
5    C:\dev\play\brainwave-data-day\2024-11-16-14-1...
6    C:\dev\play\brainwave-data-day\2024-11-16-16-1...
Name: full_filename, dtype: object